# Ввод фейковой новости, которая может оказаться фейковой

In [ ]:
str = input("Введите текст, который необходимо проверить: ")

Введите текст, который необходимо проверить: Введите текст, который необходимо проверить: Введите текст, который необходимо проверить:


In [ ]:
print(str)

Введите текст, который необходимо проверить: Введите текст, который необходимо проверить:


# Предобработка данных

In [ ]:
import re

lower_string1 = str.lower()                                                                                                 #к нижнему регистру
no_number_string1 = re.sub(r'\d+','', lower_string1)                                                                         #удаление чисел
no_wspace_string1 = no_number_string1.strip()                                                                                #удаление лишних пробелов
no_wspace_string1 = no_wspace_string1.replace(" — ", "-")
re.findall(r"^.*?(-.*?)-.*$",no_wspace_string1)                                                                              #замена
print(no_wspace_string1)

введите текст, который необходимо проверить: введите текст, который необходимо проверить:


In [ ]:
import nltk
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

tokens_without_sw1 = word_tokenize(no_wspace_string1)                                                                  #токенизация текста
print(tokens_without_sw1)

['введите', 'текст', ',', 'который', 'необходимо', 'проверить', ':', 'введите', 'текст', ',', 'который', 'необходимо', 'проверить', ':']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()                                               #лемматизация 
normal_f1 = []
for word in tokens_without_sw1:
    normal_f1.append(morph.parse(word)[0].normal_form)
print(normal_f1)

['ввести', 'текст', ',', 'который', 'необходимо', 'проверить', ':', 'ввести', 'текст', ',', 'который', 'необходимо', 'проверить', ':']


In [ ]:
from nltk.corpus import stopwords                                                                                                                                                        #удаление стоп-слов и знаков препинания
import nltk
nltk.download('stopwords')

from string import punctuation                                                                                                                                                              #проблема с фильтрацией стоп-слов - отдельная глава
punctuation

new_stopwords1 = ["это", "также", "гипотеза", "равенство", "теория", "существование", "класс", "гладкость", "уравнение", "решение"]                                                        #создание доп списка с стоп-словами 
new_punc1 = ["«", "»"]                                                                                                                                                                     #создание доп списка с пунктуационными символами
noise1 = stopwords.words('russian') + list(punctuation) 
noise1.extend(new_stopwords1) 
noise1.extend(new_punc1)
normal_f1 = [word1 for word1 in normal_f1 if not word1 in noise1] 
 
print(normal_f1)  

['ввести', 'текст', 'который', 'необходимо', 'проверить', 'ввести', 'текст', 'который', 'необходимо', 'проверить']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Алгоритм с помощью n-грамм

In [ ]:
bigrams = []
for i in range(len(normal_f1)-1):
    bigram = (normal_f1[i], normal_f1[i+1])
    bigrams.append(bigram)
    
print(bigrams)

[('ввести', 'текст'), ('текст', 'который'), ('который', 'необходимо'), ('необходимо', 'проверить'), ('проверить', 'ввести'), ('ввести', 'текст'), ('текст', 'который'), ('который', 'необходимо'), ('необходимо', 'проверить')]


# Из owl в json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rdflib import Graph
import json
g = Graph()
g.parse("/content/drive/My Drive/ontology/urn_webprotege_ontology_511a8675-2461-4b7a-ba84-e70faecb1102.owl")                                # Загружаем OWL-файл в граф RDF

json_data = g.serialize(format='json-ld', indent=4, ensure_ascii=False)                                                                    # Преобразуем граф в JSON-формат

with open("/content/drive/My Drive/ontology/owltojson.json", "w") as f:                                                                    # Сохраняем JSON-данные в файл
    json.dump(json_data, f)
print(json_data)

[
    {
        "@id": "http://webprotege.stanford.edu/R9gqE5CVDVMyWK085AGL0s",
        "@type": [
            "http://www.w3.org/2002/07/owl#Class"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "Темы новостей"
            }
        ],
        "http://www.w3.org/2000/01/rdf-schema#subClassOf": [
            {
                "@id": "http://webprotege.stanford.edu/RDqm0ObkL5xsDHWk1DaBE4Y"
            }
        ]
    },
    {
        "@id": "urn:webprotege:ontology:511a8675-2461-4b7a-ba84-e70faecb1102",
        "@type": [
            "http://www.w3.org/2002/07/owl#Ontology"
        ]
    },
    {
        "@id": "http://webprotege.stanford.edu/RDqm0ObkL5xsDHWk1DaBE4Y",
        "@type": [
            "http://www.w3.org/2002/07/owl#Class"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "Новости"
        

# Выделение маркеров фейка из json файла

In [ ]:
with open('output1.txt', 'w') as f:
    f.write(json_data)

with open('output1.txt', 'r') as f:                             # Чтение содержимого файла
    data = f.read()
                                                               # Вывод содержимого файла на экран
print(data)

[
    {
        "@id": "http://webprotege.stanford.edu/R9gqE5CVDVMyWK085AGL0s",
        "@type": [
            "http://www.w3.org/2002/07/owl#Class"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "Темы новостей"
            }
        ],
        "http://www.w3.org/2000/01/rdf-schema#subClassOf": [
            {
                "@id": "http://webprotege.stanford.edu/RDqm0ObkL5xsDHWk1DaBE4Y"
            }
        ]
    },
    {
        "@id": "urn:webprotege:ontology:511a8675-2461-4b7a-ba84-e70faecb1102",
        "@type": [
            "http://www.w3.org/2002/07/owl#Ontology"
        ]
    },
    {
        "@id": "http://webprotege.stanford.edu/RDqm0ObkL5xsDHWk1DaBE4Y",
        "@type": [
            "http://www.w3.org/2002/07/owl#Class"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "Новости"
        

In [ ]:
lower_string = data.lower()                                                                                                  #к нижнему регистру
no_number_string = re.sub(r'\d+','', lower_string)                                                                           #удаление чисел
no_wspace_string = no_number_string.strip()                                                                                   #удаление лишних пробелов
no_wspace_string = no_wspace_string.replace(" — ", "-")
re.findall(r"^.*?(-.*?)-.*$",no_wspace_string)                                                                                #замена
print(no_wspace_string)

[
    {
        "@id": "http://webprotege.stanford.edu/rgqecvdvmywkagls",
        "@type": [
            "http://www.w.org///owl#class"
        ],
        "http://www.w.org///rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "темы новостей"
            }
        ],
        "http://www.w.org///rdf-schema#subclassof": [
            {
                "@id": "http://webprotege.stanford.edu/rdqmobklxsdhwkdabey"
            }
        ]
    },
    {
        "@id": "urn:webprotege:ontology:a--ba-ba-efaecb",
        "@type": [
            "http://www.w.org///owl#ontology"
        ]
    },
    {
        "@id": "http://webprotege.stanford.edu/rdqmobklxsdhwkdabey",
        "@type": [
            "http://www.w.org///owl#class"
        ],
        "http://www.w.org///rdf-schema#label": [
            {
                "@language": "ru",
                "@value": "новости"
            }
        ]
    },
    {
        "@id": "http://webprotege.stanford.edu/r

In [ ]:
import nltk
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

tokens_without_sw = word_tokenize(no_wspace_string)                                                                  #токенизация текста
print(tokens_without_sw)

['[', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rgqecvdvmywkagls', "''", ',', '``', '@', 'type', "''", ':', '[', '``', 'http', ':', '//www.w.org///owl', '#', 'class', "''", ']', ',', '``', 'http', ':', '//www.w.org///rdf-schema', '#', 'label', "''", ':', '[', '{', '``', '@', 'language', "''", ':', '``', 'ru', "''", ',', '``', '@', 'value', "''", ':', '``', 'темы', 'новостей', "''", '}', ']', ',', '``', 'http', ':', '//www.w.org///rdf-schema', '#', 'subclassof', "''", ':', '[', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', "''", '}', ']', '}', ',', '{', '``', '@', 'id', "''", ':', '``', 'urn', ':', 'webprotege', ':', 'ontology', ':', 'a', '--', 'ba-ba-efaecb', "''", ',', '``', '@', 'type', "''", ':', '[', '``', 'http', ':', '//www.w.org///owl', '#', 'ontology', "''", ']', '}', ',', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', "''", ',', '``'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()                                                                                             #лемматизация 
normal_f = []
for word in tokens_without_sw:
    normal_f.append(morph.parse(word)[0].normal_form)
print(normal_f)

['[', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rgqecvdvmywkagls', "''", ',', '``', '@', 'type', "''", ':', '[', '``', 'http', ':', '//www.w.org///owl', '#', 'class', "''", ']', ',', '``', 'http', ':', '//www.w.org///rdf-schema', '#', 'label', "''", ':', '[', '{', '``', '@', 'language', "''", ':', '``', 'ru', "''", ',', '``', '@', 'value', "''", ':', '``', 'тема', 'новость', "''", '}', ']', ',', '``', 'http', ':', '//www.w.org///rdf-schema', '#', 'subclassof', "''", ':', '[', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', "''", '}', ']', '}', ',', '{', '``', '@', 'id', "''", ':', '``', 'urn', ':', 'webprotege', ':', 'ontology', ':', 'a', '--', 'ba-ba-efaecb', "''", ',', '``', '@', 'type', "''", ':', '[', '``', 'http', ':', '//www.w.org///owl', '#', 'ontology', "''", ']', '}', ',', '{', '``', '@', 'id', "''", ':', '``', 'http', ':', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', "''", ',', '``',

In [ ]:
from nltk.corpus import stopwords                                                                                                                                                        #удаление стоп-слов и знаков препинания
import nltk
nltk.download('stopwords')

from string import punctuation                                                                                                                                                              #проблема с фильтрацией стоп-слов - отдельная глава
punctuation

new_stopwords = ["это", "также", "гипотеза", "равенство", "теория", "существование", "класс", "гладкость", "уравнение", "решение", "//www.w.org///rdf-schema"]                                                        #создание доп списка с стоп-словами 
new_punc = ["«", "»", "''", "``", "'``"]                                                                                                                                                                     #создание доп списка с пунктуационными символами
noise = stopwords.words('russian') + list(punctuation) 
noise.extend(new_stopwords) 
noise.extend(new_punc)
normal_f = [word for word in normal_f if not word in noise] 
 
print(normal_f)  

['id', 'http', '//webprotege.stanford.edu/rgqecvdvmywkagls', 'type', 'http', '//www.w.org///owl', 'class', 'http', 'label', 'language', 'ru', 'value', 'тема', 'новость', 'http', 'subclassof', 'id', 'http', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', 'id', 'urn', 'webprotege', 'ontology', 'a', '--', 'ba-ba-efaecb', 'type', 'http', '//www.w.org///owl', 'ontology', 'id', 'http', '//webprotege.stanford.edu/rdqmobklxsdhwkdabey', 'type', 'http', '//www.w.org///owl', 'class', 'http', 'label', 'language', 'ru', 'value', 'новость', 'id', 'http', '//webprotege.stanford.edu/riuxdbrydrhpkttwresgv', 'type', 'http', '//www.w.org///owl', 'class', 'http', 'label', 'language', 'ru', 'value', 'ходжа', 'http', 'subclassof', 'id', 'http', '//webprotege.stanford.edu/rgqecvdvmywkagls', 'id', 'http', '//webprotege.stanford.edu/rcymjdrpbqdehekrwxvyej', 'type', 'http', '//www.w.org///owl', 'class', 'http', 'label', 'language', 'ru', 'value', 'риман', 'http', 'subclassof', 'id', 'http', '//webprotege.stanf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
mill_prize_problems = []

                                                                             # проходим по списку и добавляем все значения, которые идут после "value", в список
for i in range(len(normal_f)):
    if normal_f[i] == 'value':
        mill_prize_problems.append(normal_f[i+1])
                                                                                                          # выводим список с найденными значениями
mill_prize_problems.remove('тема')
mill_prize_problems.remove('новость')
mill_prize_problems.remove('p')
print(mill_prize_problems)

['ходжа', 'риман', 'бёрча-свиннертон-дайер', 'янг-миллс', 'навье-стокс']


In [ ]:
import itertools

ontology_bigrams = list(itertools.product(["доказать"], mill_prize_problems))

print(ontology_bigrams)

[('доказать', 'ходжа'), ('доказать', 'риман'), ('доказать', 'бёрча-свиннертон-дайер'), ('доказать', 'янг-миллс'), ('доказать', 'навье-стокс')]


In [ ]:
for bigram in ontology_bigrams:
    for bigram1 in bigrams:
        if bigram == bigram1:
            print('Текст вероятнее всего является фейком, так как содержит в себе шаблон "доказать" + маркер фейка. Полное совпадение найдено на маркере фейка:', bigram)
            break
    else:
        continue
    break
else:
    print('Полных совпадений не найдено')

Полных совпадений не найдено
